In [ ]:
import os
import requests
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader

# 1. Load and split PDF into chunks
pdf_path = "C://Users//admin//Downloads//narayana_annual_report_2024.pdf"  # Change to your PDF file name
loader = PyPDFLoader(pdf_path)
docs = loader.load()

splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=100)
chunks = splitter.split_documents(docs)

# 2. Create or load vector DB (Chroma)
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectordb = Chroma.from_documents(chunks, embedding=embeddings, persist_directory="./rag_db")

# 3. User query
query = input("Enter your question: ")
relevant_docs = vectordb.similarity_search(query, k=3)
context = "\n\n".join([doc.page_content for doc in relevant_docs])

# 4. Query Ollama with retrieved context + user question
prompt = f"""Use ONLY the information below to answer:\n{context}\n\nQuestion: {query}\nAnswer:"""

response = requests.post(
    "http://localhost:11434/api/generate",
    json={"model": "llama3", "prompt": prompt}
)

print("\n---\nOllama (Llama 3) response:\n")
print(response.json()["response"])
